In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train = train.drop(['Unnamed: 0', '일자'], axis=1)
test = test.drop(['Unnamed: 0', '일자'], axis=1)

In [4]:
def label_encode(train, test, columns):
    le = LabelEncoder()
    for column in columns:
        le.fit(pd.concat([train[column], test[column]]).drop_duplicates())
        train[column] = le.transform(train[column])
        test[column] = le.transform(test[column])

In [5]:
label_encode(train, test, ['요일'])

In [6]:
breakfast_train = [x for x in train['조식메뉴']]
lunch_train = [x for x in train['중식메뉴']]
dinner_train = [x for x in train['석식메뉴']]

breakfast_test = [x for x in test['조식메뉴']]
lunch_test = [x for x in test['중식메뉴']]
dinner_test = [x for x in test['석식메뉴']]

In [7]:
vectorizer = CountVectorizer()
vectorizer.fit(breakfast_test + breakfast_train)
breakfast_train = vectorizer.transform(breakfast_train).toarray()
breakfast_test = vectorizer.transform(breakfast_test).toarray()

vectorizer.fit(lunch_test + lunch_train)
lunch_train = vectorizer.transform(lunch_train).toarray()
lunch_test = vectorizer.transform(lunch_test).toarray()

vectorizer.fit(dinner_test + dinner_train)
dinner_train = vectorizer.transform(dinner_train).toarray()
dinner_test = vectorizer.transform(dinner_test).toarray()


In [8]:
train = train.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis=1)
test = test.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis=1)

In [9]:
train = pd.concat([train, pd.DataFrame(breakfast_train), pd.DataFrame(lunch_train), pd.DataFrame(dinner_train)], axis=1)
test = pd.concat([test, pd.DataFrame(breakfast_test), pd.DataFrame(lunch_test), pd.DataFrame(dinner_test)], axis=1)

In [10]:
y = train['중식계']
x = train.drop('중식계', axis=1)

In [11]:
# from sklearn.preprocessing import StandardScaler

# sc = StandardScaler()
# x = sc.fit_transform(x, y)
# test = sc.transform(test)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor(random_state=1)
model = GridSearchCV(rf, param_grid={'n_estimators': [10, 100, 1000], 'criterion': ['squared_error', 'absolute_error', 'poisson']})
model.fit(x, y)
res = model.predict(test)
submit = pd.read_csv('sample_submit.csv')
submit['중식계'] = res
submit.to_csv('submit.csv', index=False)

/Users/therealjamesjung/.pyenv/versions/miniforge3-4.10.3-10/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/therealjamesjung/.pyenv/versions/miniforge3-4.10.3-10/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/therealjamesjung/.pyenv/versions/miniforge3-4.10.3-10/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/therealjamesjung/.pyenv/versions/miniforge3-4.10.3-10/lib/python3.9/site-packages/sklearn/utils/validation.py:1858: FutureWarn

In [16]:
model.best_estimator_

RandomForestRegressor(criterion='poisson', n_estimators=1000, random_state=1)